In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
%pip install fairseq

     |████████████████████████████████| 1.7 MB 5.5 MB/s 
     |████████████████████████████████| 90 kB 3.8 MB/s 
     |████████████████████████████████| 145 kB 47.2 MB/s 
     |████████████████████████████████| 112 kB 48.6 MB/s 
     |████████████████████████████████| 74 kB 3.6 MB/s 
     |████████████████████████████████| 596 kB 45.7 MB/s 
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.8-py3-none-any.whl size=141230 sha256=efc5d60867ddaf99438424e35e8be29085a151c1b9a8dbe21f3170240680f5b4
  Stored in directory: /root/.cache/pip/wheels/ca/33/b7/336836125fc9bb4ceaa4376d8abca10ca8bc84ddc824baea6c
Successfully built antlr4-python3-runtime
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [4]:
from fairseq.models.bart import BARTModel
import torch

from tqdm import tqdm_notebook as tqdm
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
print("gpu num: ", n_gpu)

gpu num:  1


In [5]:
# I connect to my gdrive in order to get the pre trained model given by the repository
# In order to make it work I have to modifiy /usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py in load_state_dict(self, state_dict, strict)
# I have to delete the condition on unexpected keys 

from fairseq.models.bart import BARTModel
bart = BARTModel.from_pretrained(
    'gdrive/MyDrive/fairseq_multi_view/checkpoints_multi_view',
    checkpoint_file='checkpoint_best.pt',
    data_name_or_path='./cnn_dm-bin_2',
    
)
bart.eval()

RuntimeError: ignored

In [ ]:
bart.eval()
bart.cuda()

BARTHubInterface(
  (model): BARTModel(
    (encoder): TransformerEncoder(
      (dropout_module): FairseqDropout()
      (embed_tokens): Embedding(50264, 1024, padding_idx=1)
      (embed_positions): LearnedPositionalEmbedding(1026, 1024, padding_idx=1)
      (layernorm_embedding): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (layers): ModuleList(
        (0): TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (dropout_module): FairseqDropout()
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (dropout_module): FairseqDropout()
          (activation_dropout_module): FairseqDro

In [ ]:
from tqdm import tqdm as tqdm

In [ ]:
count = 1
bsz = 8
with open('gladiator_sent_trans_cons_label_2.source') as source, open('gladiator_sent_c99_label.source') as source2, open('./gladiator_best_multi_attn_best.txt', 'wt', encoding='utf-8') as fout:
    s1 = source.readlines()
    s2 = source2.readlines()

    slines = [s1[0].strip()]
    slines2 = [s2[0].strip()]

    for i in tqdm(range(1, len(s1))):
        if count % bsz == 0:
            with torch.no_grad():
                hypotheses_batch = bart.sample(slines, sentences2 = slines2, balance = True, beam=4, lenpen=2.0, max_len_b=100, min_len=5, no_repeat_ngram_size=3)
               
            for hypothesis in hypotheses_batch:
                fout.write(hypothesis + '\n')
                fout.flush()
            slines = []
            slines2 = []

        slines.append(s1[i].strip())
        slines2.append(s2[i].strip())

        count += 1

    if slines != []:

        hypotheses_batch = bart.sample(slines, sentences2 = slines2, balance = True, beam=4, lenpen=2.0, max_len_b=100, min_len=5, no_repeat_ngram_size=3)
       

        for hypothesis in hypotheses_batch:
            fout.write(hypothesis + '\n')
            fout.flush()

        

100%|██████████| 2/2 [00:00<00:00, 11932.59it/s]
/usr/local/lib/python3.7/dist-packages/fairseq/search.py:140: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beams_buf = indices_buf // vocab_size
/usr/local/lib/python3.7/dist-packages/fairseq/sequence_generator.py:651: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  unfin_idx = idx // be

In [ ]:
from rouge import Rouge, FilesRouge

In [ ]:
hyp_path = './test_best_multi_attn_best.hypo'
ref_path = './fairseq_multi_view_code/data3/test_sent_trans_cons_label.target'
hypothesis = []
with open(hyp_path, 'r') as f:
    lines = f.readlines()
    for l in lines:
        hypothesis.append(l[:-1])

reference = []
with open(ref_path, 'r') as f:
    lines = f.readlines()
    for l in lines:
        reference.append(l[:-1])

rouge = Rouge()
print('Test', rouge.get_scores(hypothesis, reference, avg = True))

Test {'rouge-1': {'f': 0.49009429017411965, 'p': 0.5079810307154228, 'r': 0.5198850300191744}, 'rouge-2': {'f': 0.25332958515740567, 'p': 0.26215165975601734, 'r': 0.2721887911613676}, 'rouge-l': {'f': 0.47529740021449296, 'p': 0.49092217200422267, 'r': 0.4978984240638189}}
